<a href="https://colab.research.google.com/github/LuFernandez/PASA/blob/master/TP2/informe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtrado óptimo

Procesamiento Adaptativo de Señales Aleatorias

© 2019 Parra, Rocío

© 2019 Fernandez, Lucero Guadalupe

In [0]:
from IPython.display import clear_output
from scipy.io.wavfile import read
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import wave, struct, math, random
from IPython.display import Audio
from scipy import signal as sgn
from scipy.linalg import toeplitz
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import ccf

In [0]:
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/sofunny.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/lorde.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/sweeplin.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/sweeplog.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/ruido.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/samples/cuadrada.wav"

!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-sofunny.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-lorde.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-sweeplin.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-sweeplog.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-ruido.wav"
!wget "https://raw.githubusercontent.com/LuFernandez/PASA/master/TP2/records/grab-cuadrada.wav"
clear_output()

In [0]:

def get_mono_array(stereo_file_name):
  file = read(stereo_file_name)
  fs = file[0]
  datos = np.array(file[1],dtype=float) 
  if len(np.shape(datos))>1: #si tiene dos canales
      ch0 = datos[:, 0]
      ch1 = datos[:, 1]
      datos = (ch0 + ch1)/2  # o lo hacemos mono
  return fs, datos
  
def get_signal(path):
  file_name = path
  fs, signal = get_mono_array(file_name)
  signal =  signal/np.linalg.norm(signal)   #normalizo
  return signal


In [0]:
#diccionario con todo lo que necesitamos, u, d, y el tiempo a correr de las señales

signals = {
  "music" : {  'u' : get_signal('./lorde.wav'),  'd' : get_signal('./grab-lorde.wav'),  'shift' : 0.555  },
  "voice" : {  'u' : get_signal('./sofunny.wav'),  'd' : get_signal('./grab-sofunny.wav'),  'shift' : 1.174  },
  "square" : {  'u' : get_signal('./cuadrada.wav'),  'd' : get_signal('./grab-cuadrada.wav'),  'shift' : 0.35  },
  "lin" : {  'u' : get_signal('./sweeplin.wav'),  'd' : get_signal('./grab-sweeplin.wav'),  'shift' : 0.45  },
  "log" : {  'u' : get_signal('./sweeplog.wav'),  'd' : get_signal('./grab-sweeplog.wav'),  'shift' : .3  },
  "noise" : {  'u' : get_signal('./ruido.wav'),  'd' : get_signal('./grab-ruido.wav'),  'shift' : 0.7  }
}
clear_output()

###Ploteamos las grabaciones y wavs originales para asegurarnos que estén bien sincronizadas

In [0]:
def sinc_signals(u, d, shift,fs,title):
  d = d[int(shift*fs):]
  N = min(len(u), len(d))
  u = u[:N]
  d = d[:N]
  plt.figure()
  plt.plot(u)
  plt.plot(d)
  plt.title(title)
  plt.show()
  return u, d

In [0]:
fs = 44100    #tanto las grabs como los originales tienen la misma frecuencia de sampleo
for sig in signals:
   signals[sig]['u'], signals[sig]['d'] = sinc_signals(signals[sig]['u'],signals[sig]['d'],signals[sig]['shift'],fs,sig)

###Calculamos r, p y la varianza de d (sigma2d)
r es el vector de los coeficientes de autocorrelación; p, el vector de correlación cruzada.

In [0]:
def estimate(u, d):
  #esta funcion no devuelve la matriz de autocorrelación R porque es de tamaño len(u), y consumiría aprox más de 666 GB de memoria
    u_fft = np.fft.fft(u)
    d_fft = np.fft.fft(d)
    
    r = np.fft.ifft(u_fft * u_fft.conj()).real
    r = r[:len(u)//2]/len(u)

    p = np.fft.ifft(u_fft * d_fft.conj()).real
    p = p[len(u)//2:]/len(u)
    p = np.flip(p)

    sigma2d = np.var(d)
    return r, p, sigma2d

In [0]:
def calc_optimum_filter(R, p, sigma2d):
  #hago la toeplitz
  #hago la inversa 
  
  return

In [0]:
R,p,sigma2d = estimate()